In [ ]:
import cv2
import dlib
import math
import tensorflow as tf
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from PIL import Image


image_paths = '/content/drive/MyDrive/SCUT-FBP5500_v2/Images'


images = []


for file_name in os.listdir(image_paths):
    file_path = os.path.join(image_paths, file_name)


    if os.path.isfile(file_path) and file_name.lower().endswith(('.jpg', '.jpeg', '.png')):

        image = Image.open(file_path)


        images.append((image,file_name))



In [ ]:
pip install dlib

In [ ]:
import dlib
import pandas as pd
from PIL import Image
import numpy as np
import math

In [ ]:
def detect_landmarks(image):

  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  faces = detector(gray)

  for face in faces:
    landmarks = predictor(gray,face)

    points = np.zeros((68,2),dtype=int)
    for i in range(0,68):
      points[i] = (landmarks.part(i).x, landmarks.part(i).y)

    return points

In [ ]:
def calculate_symmetric_ratios(landmarks):
    upper_eyebrow_numerator = np.linalg.norm(landmarks[21]-((np.linalg.norm(landmarks[22]-landmarks[21]))/2))
    upper_eyebrow_denominator = np.linalg.norm(landmarks[22]-((np.linalg.norm(landmarks[22]-landmarks[21]))/2))

    # Calculate the symmetric ratios
    lower_eyebrow_length = (np.linalg.norm(landmarks[17]-landmarks[21]))/(np.linalg.norm(landmarks[22]-landmarks[26]))
    lower_lip_length = (np.linalg.norm(landmarks[48]-landmarks[57]))/(np.linalg.norm(landmarks[54]-landmarks[57]))
    upper_eyebrow = upper_eyebrow_numerator/upper_eyebrow_denominator
    upper_lip = (np.linalg.norm(landmarks[48]-landmarks[51]))/(np.linalg.norm(landmarks[51]-landmarks[55]))
    nose = (np.linalg.norm(landmarks[31]-landmarks[33]))/(np.linalg.norm(landmarks[33]-landmarks[35]))

    return lower_eyebrow_length, lower_lip_length, upper_eyebrow, upper_lip, nose

In [ ]:
symmetric_ratios = []
counter = 1

for image, name in images:

    image_array = np.array(image)

    # Detect landmarks
    landmarks = detect_landmarks(image_array)

    # Check if landmarks were found
    if landmarks is not None:
        # Calculate golden ratios
        print("Calculated Symmetric ratios for ",counter," number of images")
        counter = counter + 1
        lower_eyebrow_length, lower_lip_length, upper_eyebrow, upper_lip, nose = calculate_symmetric_ratios(landmarks)

        # Store the ratios along with the image name
        symmetric_ratios.append({'Image Name': name, 'LEL': lower_eyebrow_length, 'LLL': lower_lip_length, 'UEB': upper_eyebrow, 'UL': upper_lip, 'Nose': nose})
    else:
        print(f"No landmarks found for image: {name}")

df = pd.DataFrame(symmetric_ratios)



In [ ]:
df

,Image Name,LEL,LLL,UEB,UL,Nose
0,AM218.jpg,0.831051,0.884666,0.902015,1.151419,0.915855
1,AM1948.jpg,0.691925,0.809862,0.921874,1.074968,0.806850
2,AM1946.jpg,0.672258,0.817819,0.895012,0.859835,0.811345
3,AM214.jpg,0.949760,1.033566,0.925908,1.253896,0.883154
4,AM1939.jpg,0.810872,0.874555,0.905741,1.345395,0.935547
...,...,...,...,...,...,...
5480,CF710.jpg,0.636479,0.822429,0.868660,0.746535,0.860621
5481,CF438.jpg,1.169349,1.114688,0.833349,1.202462,1.135649
5482,CF711.jpg,0.648047,0.679152,0.878384,0.852645,0.690510
5483,CF444.jpg,0.782817,0.836315,0.808157,0.899300,0.847585


In [ ]:
df.to_csv('/content/drive/MyDrive/symmetric_ratios.csv', index=False)
